In [ ]:
!git clone https://github.com/tensorflow/models.git

import sys
sys.path.append('/kaggle/working/models/research/object_detection/utils')
sys.path.append('/kaggle/working/models/research/object_detection/dataset_tools')

In [ ]:
import tensorflow as tf
import dataset_util
import pandas as pd
import pydicom
from io import BytesIO
import numpy as np


In [ ]:
train = pd.read_csv("../input/vinbigdata-chest-xray-abnormalities-detection/train.csv")
train = train.dropna()
train.head()

In [ ]:
train.isna().sum()

In [ ]:
train.image_id.value_counts()

In [ ]:
import numpy as np
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
import cv2
import contextlib2
import tf_record_creation_util


debug=False
def create_tf_example(imageId, boxes, class_name, class_id,  voi_lut = True, fix_monochrome = True):
    height = 1024 # Image height
    width = 1024 # Image width

    path = "../input/vinbigdata-chest-xray-abnormalities-detection/train/" + imageId + ".dicom"
    ds = pydicom.dcmread(path)
#from https://www.kaggle.com/raddar/convert-dicom-to-np-array-the-correct-way/
    if voi_lut:
        data = apply_voi_lut(ds.pixel_array, ds)
    else:
        data = ds.pixel_array
    if fix_monochrome and ds.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    ori_width, ori_height = data.shape
    data = cv2.resize(data, (width, height), interpolation = cv2.INTER_AREA)
    filename = bytes(imageId + '.jpg', 'utf-8') # Filename of the image. Empty if image is not from file
    image_format = b'jpeg' # b'jpeg' or b'png'
    if (debug):
        print(encoded_image_data[:3])

    xmins = [] # List of normalized left x coordinates in bounding box (1 per box)
    xmaxs = [] # List of normalized right x coordinates in bounding box
                # (1 per box)
    ymins = [] # List of normalized top y coordinates in bounding box (1 per box)
    ymaxs = [] # List of normalized bottom y coordinates in bounding box
                # (1 per box)

    classes_text = [] # List of string class name of bounding box (1 per box)
    classes = [] # List of integer class id of bounding box (1 per box)

    for idx, box in enumerate(boxes):
        if not np.isnan(box[0]):
            if (debug):
                print(box)
            classes_text.append(str(class_name[idx]).encode())
            classes.append(int(class_id[idx]))
            
            # x-min y-min width height
            xmins.append(box[0] *width/ori_width)   # store normalized values for bbox
            xmaxs.append(box[2] * width/ori_width)
            ymins.append(box[1] * height/ori_height)
            ymaxs.append(box[3] * height/ori_height)

    if (debug):
        print(xmins)
        print(xmaxs)
        print(ymins)
        print(ymaxs)
    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.float_list_feature(data.tostring()),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example


num_shards=10
output_filebase='train'

with contextlib2.ExitStack() as tf_record_close_stack:
    output_tfrecords = tf_record_creation_util.open_sharded_output_tfrecords(tf_record_close_stack, output_filebase, num_shards)
    groups = train.groupby('image_id')

    count = 0

    for image_id in train.drop_duplicates('image_id')['image_id']:
        print('[{c}]processing patientId = {p}'.format(c=count,p=image_id))

        boxes = groups.get_group(image_id)[['x_min','y_min','x_max','y_max']].values
        class_name = groups.get_group(image_id)[['class_name']].values
        class_id = groups.get_group(image_id)[['class_id']].values
        tf_example = create_tf_example(image_id, boxes, class_name, class_id)

        output_shard_index = count % num_shards
        output_tfrecords[output_shard_index].write(tf_example.SerializeToString())
        if debug:
            if count>10:
                break
        count += 1